In [7]:
import time
from collections import defaultdict
from itertools import combinations
import numpy as np
import pandas as pd
df = pd.read_csv('distance_matrix.csv')
import gurobipy as gp
from gurobipy import GRB

def shortest_subtour(edges):
    """Given a list of edges, return the shortest subtour (as a list of nodes)
    found by following those edges. It is assumed there is exactly one 'in'
    edge and one 'out' edge for every node represented in the edge list."""

    # Create a mapping from each node to its neighbours
    node_neighbors = defaultdict(list)
    for i, j in edges:
        node_neighbors[i].append(j)
    assert all(len(neighbors) == 2 for neighbors in node_neighbors.values())

    # Follow edges to find cycles. Each time a new cycle is found, keep track
    # of the shortest cycle found so far and restart from an unvisited node.
    unvisited = set(node_neighbors)
    shortest = None
    while unvisited:
        cycle = []
        neighbors = list(unvisited)
        while neighbors:
            current = neighbors.pop()
            cycle.append(current)
            unvisited.remove(current)
            neighbors = [j for j in node_neighbors[current] if j in unvisited]
        #if shortest is None or len(cycle) < len(shortest):
            shortest = cycle

    assert shortest is not None
    return shortest


class TSPCallback:
    """Callback class implementing lazy constraints for the TSP.  At MIPSOL
    callbacks, solutions are checked for subtours and subtour elimination
    constraints are added if needed."""

    def __init__(self, nodes, x):
        self.nodes = nodes
        self.x = x

    def __call__(self, model, where):
        """Callback entry point: call lazy constraints routine when new
        solutions are found. Stop the optimization if there is an exception in
        user code."""
        if where == GRB.Callback.MIPSOL:
            try:
                self.eliminate_subtours(model)
            except Exception:
                logging.exception("Exception occurred in MIPSOL callback")
                model.terminate()

    def eliminate_subtours(self, model):
        """Extract the current solution, check for subtours, and formulate lazy
        constraints to cut off the current solution if subtours are found.
        Assumes we are at MIPSOL."""
        values = model.cbGetSolution(self.x)
        edges = [(i, j) for (i, j), v in values.items() if v > 0.5]
        tour = shortest_subtour(edges)
        if len(tour) < len(self.nodes):
            # add subtour elimination constraint for every pair of cities in tour
            model.cbLazy(
                gp.quicksum(self.x[i, j] for i, j in combinations(tour, 2))
                <= len(tour) - 1
            )


def solve_tsp(nodes, distances):
    """
    Solve a dense symmetric TSP using the following base formulation:

    min  sum_ij d_ij x_ij
    s.t. sum_j x_ij == 2   forall i in V
         x_ij binary       forall (i,j) in E

    and subtours eliminated using lazy constraints.
    """

    with gp.Env() as env, gp.Model(env=env) as m:
        # Create variables, and add symmetric keys to the resulting dictionary
        # 'x', such that (i, j) and (j, i) refer to the same variable.
        x = m.addVars(distances.keys(), obj=distances, vtype=GRB.BINARY, name="e")
        x.update({(j, i): v for (i, j), v in x.items()})

        # Create degree 2 constraints
        for i in nodes:
            m.addConstr(gp.quicksum(x[i, j] for j in nodes if i != j) == 2)

        # Optimize model using lazy constraints to eliminate subtours
        m.Params.LazyConstraints = 1
        cb = TSPCallback(nodes, x)
        m.optimize(cb)

        # Extract the solution as a tour
        edges = [(i, j) for (i, j), v in x.items() if v.X > 0.5]
        print(len(edges))
        tour = shortest_subtour(edges)
        assert set(tour) == set(nodes)

        return tour, m.ObjVal
if __name__ == "__main__":
    # Parse arguments

    npoints = 34

    # Create n random points in 2D
    nodes = list(range(npoints))

    # Dictionary of Euclidean distance between each pair of points
    distances = {
        (i, j):  round(df.iloc[i,j])
        for i, j in combinations(nodes, 2)
    }
    time_start = time. time()
    tour, cost = solve_tsp(nodes, distances)
    time_end = time. time()

    print("")
    print(f"Optimal tour: {tour}")
    print(f"Optimal cost: {cost:g}")
    print(f"Time cost: {(time_end-time_start):.3f}")
    print("")

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 34 rows, 561 columns and 1122 nonzeros
Model fingerprint: 0xaf415b38
Variable types: 0 continuous, 561 integer (561 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 45444.000000
Presolve time: 0.00s
Presolved: 34 rows, 561 columns, 1122 nonzeros
Variable types: 0 continuous, 561 integer (561 binary)

Root relaxation: objective 1.409950e+04, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

In [2]:
routine = pd.DataFrame(columns=["city","lat", "lon"])
df_start = pd.read_csv('cityinfo.csv')

for i in range(34):
    routine.loc[i] = df_start.loc[tour[i]]

import folium

# 创建一个中心位于北京的地图
m = folium.Map(location=[39.9042, 116.4074], zoom_start=5)

# 假设我们有一个包含城市坐标的数据集
cities_data = routine

# 添加城市标记
for index, row in cities_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['city'],
        icon=folium.Icon(color='red')
    ).add_to(m)

# 连接城市
temp = cities_data[['lat', 'lon']].values.tolist()
locations = np.vstack((temp, temp[0]))
folium.PolyLine(locations, color="blue", weight=2.5, opacity=1).add_to(m)

m.save('routine_gurobi.html')

In [3]:
set({'a':1,'b':2})

{'a', 'b'}